**08-04 케라스의 SimpleRNN과 LSTM 이해하기**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

In [13]:
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype = np.float32)
print(np.shape(train_X))

(1, 4, 5)


In [14]:
rnn = SimpleRNN(3, return_sequences = False, return_state = False)
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[ 0.70021546 -0.8776991   0.8160635 ]], shape: (1, 3)


In [15]:
rnn = SimpleRNN(3, return_sequences = True, return_state = False)
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[[ 0.9719573   0.94384956 -0.83806473]
  [ 0.9793657   0.86449057 -0.9966508 ]
  [ 0.8859479   0.85704637 -0.99077713]
  [ 0.99080545  0.99034834 -0.9792287 ]]], shape: (1, 4, 3)


In [17]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) +1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [18]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [19]:
sequences = list()
for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[: i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [20]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [21]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [22]:
sequences = pad_sequences(sequences, maxlen=max_len, padding ='pre')

In [24]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [26]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [27]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [28]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [29]:
y = to_categorical(y, num_classes = vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

embedding_dim = 10
hidden_units =32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs=200, verbose =2)

In [32]:
def sentence_generation(model, tokenizer, current_word, n): #모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    for _ in range(n):
        #현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X에 대해서 Y를 예측하고 Y를 result에 저장
        result = model.predict(encoded, verbose = 0)
        result = np.argmax(result, axis =1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        current_word = current_word + " "+word
        sentence = sentence + ' ' + word
    sentence = init_word +sentence
    return sentence

In [33]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [34]:
#LSTM을 이용하여 텍스트 생성하기

# 데이터에 대한 이해와 전처리

import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [35]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [36]:
print(df['headline'].isnull().values.any())

False


In [38]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [39]:
print('총 샘플의 개수: {}'.format(len(headline)))

총 샘플의 개수: 1324


In [40]:
headline = [word for word in headline if word != "Unknown"]
print('노이즈값 제거 후 샘플의 개수:{}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수:1214


In [41]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [44]:
def repreprocessing(raw_sentence):
    preprocessed_sentence = raw_sentence.encode("utf8").decode("ascii", 'ignore')
    return "".join(word for word in preprocessed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) +1
print('단어 집합의 크기 :%d' % vocab_size)

단어 집합의 크기 :3494


In [47]:
sequences = list()

for sentence in preprocessed_headline:
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [48]:
index_to_word ={}
for key, value in tokenizer.word_index.items():
    index_to_word[value] =key
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [49]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [50]:
sequences = pad_sequences(sequences, maxlen=max_len, padding = 'pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [51]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [52]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [53]:
print(y[:3])

[ 269  371 1115]


In [54]:
y = to_categorical(y, num_classes = vocab_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 200, verbose=2)

In [56]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            if index == result:
                break

        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word

    sentence = init_word +sentence
    return sentence

In [57]:
print()

**08-07 문자 단위 RNN**

In [1]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

#데이터 로드
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename='11-0.txt')

f = open('11-0.txt','rb')
sentences = []
for sentence in f:
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.decode('ascii','ignore')

    if len(sentence)>0:
        sentences.append(sentence)

f.close()

In [2]:
sentences[:5]

["*** start of the project gutenberg ebook alice's adventures in",
 'wonderland ***',
 '[illustration]',
 'alices adventures in wonderland',
 'by lewis carroll']

In [3]:
sentences.index('down the rabbit-hole')
print(sentences[20])
sentences = sentences[20:]
print(sentences[:5])

down the rabbit-hole
['down the rabbit-hole', 'alice was beginning to get very tired of sitting by her sister on the', 'bank, and of having nothing to do: once or twice she had peeped into', 'the book her sister was reading, but it had no pictures or', 'conversations in it, and what is the use of a book, thought alice']


In [4]:
total_data = ' '.join(sentences)
print('문자열의 길이 또는 총 문자의 개수: %d' % len(total_data))

문자열의 길이 또는 총 문자의 개수: 139678


In [5]:
print(total_data[:200])

down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it 


In [6]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print('문자 집합의 크기: {}'.format(vocab_size))

문자 집합의 크기: 41


In [7]:
#문자에 고유한 정수 부여
char_to_index = dict((char,index) for index, char in enumerate(char_vocab))
print('문자 집합 :', char_to_index)

문자 집합 : {' ': 0, '!': 1, "'": 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, '[': 12, ']': 13, '_': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'k': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39, 'z': 40}


In [8]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [9]:
train_X = 'appl'
train_y = 'pple'

In [10]:
seq_length = 60

#문자열의 길이를 seq_length로 나누면 전처리 후 생겨날 샘플 수
n_samples = int(np.floor((len(total_data)-1) / seq_length))
print('샘플의 수 : {}'.format(n_samples))

샘플의 수 : 2327


In [11]:
#전처리
train_X =[]
train_y =[]

for i in range(n_samples):
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 pick.
    X_sample = total_data[i*seq_length: (i+1)* seq_length]

    #정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트
    y_sample = total_data[i*seq_length +1: (i+1)*seq_length+1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [12]:
print('X데이터의 첫번째 샘플:', train_X[0])
print('y데이터의 첫번째 샘플:', train_y[0])
print('-'*50)
print('X 데이터의 첫번째 샘플 디코딩 :', [index_to_char[i] for i in train_X[0]])
print('y 데이터의 첫번째 샘플 디코딩 :', [index_to_char[i] for i in train_y[0]])

X데이터의 첫번째 샘플: [18, 29, 37, 28, 0, 34, 22, 19, 0, 32, 15, 16, 16, 23, 34, 7, 22, 29, 26, 19, 0, 15, 26, 23, 17, 19, 0, 37, 15, 33, 0, 16, 19, 21, 23, 28, 28, 23, 28, 21, 0, 34, 29, 0, 21, 19, 34, 0, 36, 19, 32, 39, 0, 34, 23, 32, 19, 18, 0, 29]
y데이터의 첫번째 샘플: [29, 37, 28, 0, 34, 22, 19, 0, 32, 15, 16, 16, 23, 34, 7, 22, 29, 26, 19, 0, 15, 26, 23, 17, 19, 0, 37, 15, 33, 0, 16, 19, 21, 23, 28, 28, 23, 28, 21, 0, 34, 29, 0, 21, 19, 34, 0, 36, 19, 32, 39, 0, 34, 23, 32, 19, 18, 0, 29, 20]
--------------------------------------------------
X 데이터의 첫번째 샘플 디코딩 : ['d', 'o', 'w', 'n', ' ', 't', 'h', 'e', ' ', 'r', 'a', 'b', 'b', 'i', 't', '-', 'h', 'o', 'l', 'e', ' ', 'a', 'l', 'i', 'c', 'e', ' ', 'w', 'a', 's', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'i', 'n', 'g', ' ', 't', 'o', ' ', 'g', 'e', 't', ' ', 'v', 'e', 'r', 'y', ' ', 't', 'i', 'r', 'e', 'd', ' ', 'o']
y 데이터의 첫번째 샘플 디코딩 : ['o', 'w', 'n', ' ', 't', 'h', 'e', ' ', 'r', 'a', 'b', 'b', 'i', 't', '-', 'h', 'o', 'l', 'e', ' ', 'a', 'l', 'i', 'c',

In [13]:
#원-핫 인코딩 / 임베딩층을 사용하지 않을것이므로 입력 시퀀스에도 원-핫 인코딩 수행
train_X = to_categorical(train_X) #입력 시퀀스에도 원-핫 인코딩 수행
train_y = to_categorical(train_y)

print('train_X의 크기(shape) :{}'.format(train_X.shape))
print('train_y의 크기(shape) :{}'.format(train_y.shape))

#(샘플의수, 입력시퀀스의 길이, 벡터의 차원)

train_X의 크기(shape) :(2327, 60, 41)
train_y의 크기(shape) :(2327, 60, 41)


In [14]:
#모델 설계하기

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences= True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

#예측하여 문장을 생성하는 함수 정의

def sentence_generation(model, length):
    # 문자에 대한 랜덤한 정수 생성
    ix = [np.random.randint(vocab_size)]

    #랜덤한 정수로부터 맵핑되는 문자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1], '번 문자', y_char[-1],'로 예측을 시작!')

    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end='')
        ix = np.argmax(model.predict(X[:,:i+1,:])[0],1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

result = sentence_generation(model, 100)
print(result)

Epoch 1/80
73/73 - 11s - loss: 3.0303 - accuracy: 0.1859 - 11s/epoch - 144ms/step
Epoch 2/80
73/73 - 1s - loss: 2.6931 - accuracy: 0.2556 - 604ms/epoch - 8ms/step
Epoch 3/80
73/73 - 1s - loss: 2.3197 - accuracy: 0.3430 - 602ms/epoch - 8ms/step
Epoch 4/80
73/73 - 1s - loss: 2.1669 - accuracy: 0.3829 - 607ms/epoch - 8ms/step
Epoch 5/80
73/73 - 1s - loss: 2.0546 - accuracy: 0.4114 - 606ms/epoch - 8ms/step
Epoch 6/80
73/73 - 1s - loss: 1.9669 - accuracy: 0.4325 - 612ms/epoch - 8ms/step
Epoch 7/80
73/73 - 1s - loss: 1.8952 - accuracy: 0.4504 - 613ms/epoch - 8ms/step
Epoch 8/80
73/73 - 1s - loss: 1.8297 - accuracy: 0.4681 - 599ms/epoch - 8ms/step
Epoch 9/80
73/73 - 1s - loss: 1.7744 - accuracy: 0.4827 - 610ms/epoch - 8ms/step
Epoch 10/80
73/73 - 1s - loss: 1.7241 - accuracy: 0.4958 - 597ms/epoch - 8ms/step
Epoch 11/80
73/73 - 1s - loss: 1.6779 - accuracy: 0.5073 - 607ms/epoch - 8ms/step
Epoch 12/80
73/73 - 1s - loss: 1.6328 - accuracy: 0.5194 - 604ms/epoch - 8ms/step
Epoch 13/80
73/73 - 1s -